# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686529


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:24,  3.00s/it]

Rendering models:  10%|█         | 3/30 [00:08<01:14,  2.74s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:51,  2.00s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:43,  1.75s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:24<00:02,  1.34it/s]

Rendering models:  90%|█████████ | 27/30 [00:25<00:02,  1.34it/s]

Rendering models:  93%|█████████▎| 28/30 [00:26<00:01,  1.23it/s]

Rendering models:  97%|█████████▋| 29/30 [00:27<00:00,  1.22it/s]

Meckert7597                           0.000288
Blueliz13                             0.000495
smader24                              0.013938
not-logged-in-f6fac0763296d5759c01    0.000122
Dynamatt                              0.000081
SavinSpaceEngineer                    0.000071
djswanso                              0.000145
Roquwat                               0.001935
mtajnai                               0.001618
jmfranci                              0.000125
browny811                             0.000117
lsautter                              0.000107
TheEpicPrimius123                     0.007675
TRISTANM18                            0.000182
Lavadude                              0.007585
ShirleyL                              0.000118
not-logged-in-5511d3382f7100ebd4e1    0.000148
not-logged-in-8700ff8251e0704f745c    0.000084
equidad1                              0.000086
wgoltz                                0.000079
onustu                                0.000150
joshmackwilli

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())